In [1]:
from dask.distributed import Client
import os

import dask

If you have an existing Dask cluster running already, set the scheduler address below. Otherwise, leave it to `None` and a local cluster will be created.

In [2]:
scheduler_address = None #"tcp://10.2.168.161:8786"

if scheduler_address is None:
    from dask_cuda import LocalCUDACluster
    cluster = LocalCUDACluster()
    c = Client(cluster)
else:
    c = Client(scheduler_address)
c

/raid/cjnolet/cuml5/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:38984 Dashboard: http://127.0.0.1:43984/status,Cluster Workers: 8 Cores: 8 Memory: 540.95 GB


In [6]:
from nccl_example import nccl, inject_comms_on_handle, SimpleReduce
from cuml.common.handle import Handle

import random
from dask.distributed import wait

In [4]:
from dask import delayed
import dask.dataframe as dd
import numba.cuda
import cudf
import numpy as np

In [5]:
from tornado import gen
from dask.distributed import default_client
from toolz import first
import logging
import dask.dataframe as dd

import dask_cudf
import numpy as np
import cudf
import pandas as pd

from dask.distributed import wait


def parse_host_port(address):
    if '://' in address:
        address = address.rsplit('://', 1)[1]
    host, port = address.split(':')
    port = int(port)
    return host, port

import dask_cudf


@gen.coroutine
def extract_ddf_partitions(ddf):
    """
    Given a Dask cuDF, return a tuple with (worker, future) for each partition
    """
    client = default_client()
    
    delayed_ddf = ddf.to_delayed()
    parts = client.compute(delayed_ddf)
    yield wait(parts)
    
    key_to_part_dict = dict([(str(part.key), part) for part in parts])
    who_has = yield client.who_has(parts)

    worker_map = []
    for key, workers in who_has.items():
        worker = parse_host_port(first(workers))
        worker_map.append((worker, key_to_part_dict[key]))

    gpu_data = [(worker, part) for worker, part in worker_map]

    yield wait(gpu_data)

    raise gen.Return(gpu_data)
    
    
def create_df(f, m, n):
    """
    Generates a cudf of the given size with all values initialized to 1 
    """
    X = np.ones((m, n), dtype = np.float32)
    ret = cudf.DataFrame([(i,
                           X[:, i].astype(np.float32)) for i in range(n)],
                         index=cudf.dataframe.RangeIndex(f * m,
                                                         f * m + m, 1))
    return ret

def get_meta(df):
    ret = df.iloc[:0]
    return ret

def gen_dask_cudf(nrows, ncols):
    workers = c.has_what().keys()

    # Create dfs on each worker (gpu)
    dfs = [c.submit(create_df, n, nrows, ncols, workers=[worker])
           for worker, n in list(zip(workers, list(range(len(workers)))))]
    # Wait for completion
    wait(dfs)

    meta = c.submit(get_meta, dfs[0]).result()
    return dask_cudf.from_delayed(dfs, meta=meta)

In [7]:
from cuml.cluster import KMeans

class Dask_NCCL_Demo:
    
    def __init__(self, client):
        self.client = client
    
    
    @staticmethod
    def func_init_(workerId, nWorkers, uniqueId):
        """
        Initialize ncclComm_t on worker
        """
        w = dask.distributed.get_worker()

        # 1. initialize any necessary comms on each worker
        n = nccl()
        n.init(nWorkers, uniqueId, workerId)

        # 2. Initialize cumlCommunicator and inject into cumlHandle
        handle = Handle()
        inject_comms_on_handle(handle, n, nWorkers, workerId)

        # 3. Use the cumlHandle w/ cumlCommunicator for model
        a = KMeans(handle = handle)
        #a = SimpleReduce(workerId, nWorkers, n.get_comm())

        return (n, a)
    
    @staticmethod
    def func_get_size_(cuml_comm, r):
        """
        Runs on worker: test the underlying comm by getting 
        the clique size 
        """
        return cuml_comm[1].get_clique_size()
    
    @staticmethod
    def func_get_rank_(cuml_comm, r):
        """
        Runs on worker: test the underlying comm by getting
        the rank of the worker
        """
        return cuml_comm[1].get_rank()
    
    @staticmethod
    def get_meta(df):
        return df.iloc[:0]
    
    @staticmethod
    def func_fit_transform(model, df, root_rank, r):
        """
        This function is executed on the workers and performs the necessary
        data preparation, as well as calling the cython-wrapped C++ "algorithm"
        function(s), returning a cuDF with results, if necessary. 
        
        The client will construct a Dask cuDF out of the cuDFs returned from
        this function, if necessary. 
        """

        nccl_comm, model = model
        
        # Execute our cython-wrapped C++ "algorithm"
        return model.fit(df).transform(df)

    def fit_transform(self, X):
        """
        An end to end example to mimic a typical Dask cuML algorithm using
        OPG semantics. This function is executed on the client.
        
        The following steps are taken with Dask cuDF as input:
        1. Co-locate Dask cuDF partitions with nccl communicator and our demo "model" on each worker
        2. Run the algorithm, extracting the Numba device memory pointer for each partition
           and allocating necessary output memory on device for constructing the cuDF 
           partition(s) that will be returned to the user. 
        3. Construct Dask cuDF from the futures containing the cuDFs returned from local 
           "algorithm" functions on each worker. 
        """
        
        # Keep the futures around so the GPU memory doesn't get
        # deallocated on the workers.
        gpu_futures = c.sync(extract_ddf_partitions, X)

        worker_model_map = dict(map(lambda x: (x[1], x[2]), self.clique))

        # Run our "algorithm" to perform reduce 
        f = [c.submit(Dask_NCCL_Demo.func_fit_transform, # Function to run on worker
                      worker_model_map[w],               # tuple(nccl_comm, SimpleReduce instance)
                      f,                                 # Input DataFrame partition
                      0,                                 # Root rank
                      random.random())                   # Makes sure all workers call function 
             for w, f in gpu_futures] 
        wait(f)
        
        # Convert result back into a dask_cudf
        dfs = [d for d in f if d.type != type(None)]
        meta = c.submit(Dask_NCCL_Demo.get_meta, dfs[0]).result()
        ddf = dd.from_delayed(dfs, meta=meta)
        
        return ddf
    
    def worker_ranks(self):
        """
        Builds a dictionary of { (worker_address, worker_port) : worker_rank }
        """
        return dict(list(map(lambda x: (x[1], x[0]), self.clique)))

    def run_func_on_workers(self, func):
        """
        Simple helper function to schedule a function on all workers
        of a clique
        """
        f = [c.submit(func, a, random.random()) for i, w, a in self.clique]
        wait(f)
        return [a.result() for a in f]

    def get_workers_(self):
        """
        Return the list of workers parsed as [(address, port)]
        """
        return list(map(lambda x: parse_host_port(x), self.client.has_what().keys()))
    
    def init(self, uniqueId):
        """
        Use nccl-py to initialize ncclComm_t on each worker and 
        store the futures for this instance. 
        """
        workers = self.get_workers_()
        workers_indices = list(zip(workers, range(len(workers))))

        self.clique = [(idx, worker, self.client.submit(Dask_NCCL_Demo.func_init_, 
                                           idx, 
                                           len(workers), 
                                           uniqueId,
                                           workers=[worker]))
             for worker, idx in workers_indices]
        
    def get_clique_size(self):
        return self.run_func_on_workers(Dask_NCCL_Demo.func_get_size_)

    def get_rank(self):
        return self.run_func_on_workers(Dask_NCCL_Demo.func_get_rank_)
    

First demonstration will show that NCCL works within Dask as expected. This includes:
1. Creation of a NCCL clique using Dask workers to broadcast the ncclUniqueId
2. Demonstrating the NCCL clique successfully performs collective comms
3. All calls to the underlying NCCL comm are made through the cuML comms facade. 

In [8]:
world_id = nccl.get_unique_id()

In [9]:
demo = Dask_NCCL_Demo(c)
demo.init(world_id)

In [10]:
demo.get_clique_size()

AttributeError: 'KMeans' object has no attribute 'get_clique_size'

In [10]:
demo.get_rank()

[0, 1, 2, 3, 4, 5, 6, 7]

In [11]:
demo.worker_ranks()

{('127.0.0.1', 34250): 0,
 ('127.0.0.1', 35129): 1,
 ('127.0.0.1', 35869): 2,
 ('127.0.0.1', 37461): 3,
 ('127.0.0.1', 38195): 4,
 ('127.0.0.1', 42525): 5,
 ('127.0.0.1', 44419): 6,
 ('127.0.0.1', 46415): 7}

The second demonstration will show that we can take a Dask cuDF and utilize NCCL on the workers hosting its partitions. This includes:
1. Initializing a NCCL clique with the workers that host partitions underlying the Dask cuDF
2. Perform a collective comm operation across the partitions using only the cuML comms facade to communicate with the NCCL comm. 
3. Output a Dask cuDF with the results of the collective comm. 

In [11]:
X = gen_dask_cudf(10, 50)

In [12]:
c.has_what()

{'tcp://127.0.0.1:32861': ('func_init_-a71c233aee224fc26dbb5814fbb430b5',
  'create_df-a197a3749394220cdb46b22d7e22e5aa'),
 'tcp://127.0.0.1:36206': ('create_df-b486209d0d3d8d405fd3f412af4d93dd',
  'func_init_-4c59850e03537d05ee83232f7ea71c4d'),
 'tcp://127.0.0.1:36896': ('create_df-a6c56a9a8a51cf154cee198e492c9d87',
  'func_init_-67a05abe09e32437147498edef0baf67'),
 'tcp://127.0.0.1:38656': ('func_init_-9816649ccd06b4956e11d95970db14c1',
  'create_df-26043e82cb25e74bf26ec2a3f625642f'),
 'tcp://127.0.0.1:39491': ('create_df-2df214e766a1dbf9480e3fbeddb84ac6',
  'func_init_-96945057e95fe70f778bcb12808f299a'),
 'tcp://127.0.0.1:41502': ('func_init_-6ba15b369e688198c7fd32037738c942',
  'create_df-731fa1c943ef87ae06019d6b89065f7a'),
 'tcp://127.0.0.1:45176': ('create_df-3525375ac9aeb05d87b868388a97edd4',
  'func_init_-a55726ee6c1f5fddf0be5108813d4a22'),
 'tcp://127.0.0.1:45850': ('func_init_-7683987ccfbd6e8c534b625ded2bc5df',
  'create_df-cebb128559a98cf04b31d750867db360')}

In [14]:
result = demo.fit_transform(X)

In [15]:
print(str(result))

<dask_cudf.DataFrame | 2 tasks | 1 npartitions>


In [16]:
print(str(result.compute()))

     0    1    2    3    4    5    6 ...   49
0  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
1  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
2  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
3  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
4  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
5  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
6  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
7  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
8  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
9  8.0  8.0  8.0  8.0  8.0  8.0  8.0 ...  8.0
[42 more columns]
